In [3]:

import pandas as pd
import numpy as np


In [4]:

# Leer dataframes
df_clientes = pd.read_csv('../Datasets/olist_customers_dataset.csv')
df_pedidos = pd.read_csv('../Datasets/olist_orders_dataset.csv')
df_productos_en_pedidos = pd.read_csv('../Datasets/olist_order_items_dataset.csv')
df_productos = pd.read_csv('../Datasets/olist_products_dataset.csv')
df_calificaciones = pd.read_csv('../Datasets/olist_order_reviews_dataset.csv')


In [3]:

df_clientes.dtypes


customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [26]:

df_pedidos.dtypes


order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [28]:

df_productos_en_pedidos.dtypes


order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [30]:

df_productos.dtypes


product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [47]:

df_calificaciones.dtypes


review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [5]:

# Join para agregar a Productos información del pedidos
df_productos_comprados = pd.merge(
    df_productos_en_pedidos[['order_id','product_id','price','freight_value']],
    df_productos[['product_id','product_category_name']], on=['product_id'])


In [6]:

# Join para agrefar la informacón del pedido
df_pedidos_prodcutos = pd.merge(df_productos_comprados, df_pedidos, on=['order_id'])


In [7]:

# Join para agregar la información del cliente
df_pedidos_clientes = pd.merge(
    df_pedidos_prodcutos,
    df_clientes[['customer_id','customer_city','customer_state']],
    on=['customer_id'])


In [8]:

# Join para agregar las calificaciones
df_ventas = pd.merge(df_pedidos_clientes,df_calificaciones[['order_id','review_score']], on=['order_id'])


In [9]:

# Depurar y afinar df_ventas
df_ventas = df_ventas.drop(['product_id','customer_id'], axis=1)
df_ventas = df_ventas.rename(columns={'order_id':'Id_Pedidos',
                                      'price':'Precio_Producto',  
                                      'freight_value':'Precio_Flete',
                                      'product_category_name':'Categoria_Producto',
                                      'order_status':'Estatus_Pedido',
                                      'order_purchase_timestamp':'FH_Pedido',
                                      'order_approved_at':'FH_Aprovacion',
                                      'order_delivered_carrier_date':'FH_Envio',
                                      'order_delivered_customer_date':'FH_Entrega',
                                      'order_estimated_delivery_date':'F_Estimada_Entrega',
                                      'customer_city':'Ciudad',
                                      'customer_state':'Estado',
                                      'review_score':'Calificacion'})


In [10]:

# Convertir el tipo de datos de las variables de fecha, de Object a Datetime
df_ventas['FH_Pedido'] = df_ventas['FH_Pedido'].astype('datetime64')
df_ventas['FH_Aprovacion'] = df_ventas['FH_Aprovacion'].astype('datetime64')
df_ventas['FH_Envio'] = df_ventas['FH_Envio'].astype('datetime64',errors='ignore')
df_ventas['FH_Entrega'] = df_ventas['FH_Entrega'].astype('datetime64',errors='ignore')
df_ventas['F_Estimada_Entrega'] = df_ventas['F_Estimada_Entrega'].astype('datetime64',errors='ignore')


In [12]:

# Exportar dataframe
df_ventas.to_feather('../Datasets/Ventas.feather')
df_ventas.to_csv('../Datasets/Ventas.csv', index=False, encoding='utf-8')
